In [87]:
import csv
from datetime import datetime
import json
from Queue import Queue
import string
from threading import Thread

from ipywidgets import FloatProgress
from IPython.display import display
import numpy as np
import pandas as pd

# from creds import apikey

In [88]:
from __future__ import print_function

import argparse
import pprint
import requests
import sys
import urllib

try:
    # For Python 3.0 and later
    from urllib.error import HTTPError
    from urllib.parse import quote
    from urllib.parse import urlencode
except ImportError:
    # Fall back to Python 2's urllib2 and urllib
    from urllib2 import HTTPError
    from urllib import quote
    from urllib import urlencode

In [89]:
# Need to import the list of zip codes to search
data = pd.read_csv('postal_code.csv')

# Need to add '0' and '00' to shorter zip codes
data['zip'] = data['zip'].astype(str)

# Create array for just zip codes
zip_code = []

for row in range(0,len(data['zip'])):
    zip_code.append(data['zip'][row])

In [90]:
for row in range(0,len(zip_code)):
    if len(zip_code[row]) == 4:
        zip_code[row] = '0' + zip_code[row]
    elif len(zip_code[row]) == 3:
        zip_code[row] = '00' + zip_code[row]
    else:
        zip_code[row] = zip_code[row]

In [91]:
zip_code

['16820',
 '36310',
 '31001',
 '70510',
 '70511',
 '38601',
 '29620',
 '04406',
 '54405',
 '76621',
 '17301',
 '67510',
 '20606',
 '58001',
 '83210',
 '42201',
 '21001',
 '39730',
 '28315',
 '45101',
 '57401',
 '57402',
 '98520',
 '21005',
 '35440',
 '79311',
 '68001',
 '67410',
 '79601',
 '79602',
 '79603',
 '79604',
 '79605',
 '79606',
 '79608',
 '79697',
 '79698',
 '79699',
 '61410',
 '21009',
 '24210',
 '24211',
 '24212',
 '06230',
 '02351',
 '19001',
 '87510',
 '70420',
 '54101',
 '58002',
 '59001',
 '08205',
 '08201',
 '95220',
 '21520',
 '20607',
 '23301',
 '02018',
 '12404',
 '25606',
 '77326',
 '74720',
 '23001',
 '79713',
 '39735',
 '18010',
 '50601',
 '50001',
 '71316',
 '49610',
 '15610',
 '98220',
 '15520',
 '12405',
 '93510',
 '01720',
 '04001',
 '56432',
 '59002',
 '02743',
 '30101',
 '30102',
 '03601',
 '49355',
 '49356',
 '49357',
 '49301',
 '56510',
 '45810',
 '74520',
 '74820',
 '74821',
 '15410',
 '50002',
 '61411',
 '74330',
 '30103',
 '42202',
 '99546',
 '05640',


In [99]:
# API constants
API_HOST = 'https://api.yelp.com'
SEARCH_PATH = '/v3/businesses/search'
BUSINESS_PATH = '/v3/businesses/'  # Business ID will come after slash.
TOKEN_PATH = '/oauth2/token'
GRANT_TYPE = 'client_credentials'

DEFAULT_TERM = 'breweries'
DEFAULT_LOCATION = '94596'
LOCATIONS = zip_code
SEARCH_LIMIT = 20

CLIENT_ID = '1zNgmss9HHwbHm0N00zcrA'
CLIENT_SECRET = 'n6XUW0fJbPMQXCqq7gr4lWUoTvbx1XwsakVCE2AAhmCvNEMFolKyVZdirnzfPCbC'

In [100]:
def obtain_bearer_token(host, path):
    """Given a bearer token, send a GET request to the API.
    Args:
        host (str): The domain host of the API.
        path (str): The path of the API after the domain.
        url_params (dict): An optional set of query parameters in the request.
    Returns:
        str: OAuth bearer token, obtained using client_id and client_secret.
    Raises:
        HTTPError: An error occurs from the HTTP request.
    """
    url = '{0}{1}'.format(host, quote(path.encode('utf8')))
    assert CLIENT_ID, "Please supply your client_id."
    assert CLIENT_SECRET, "Please supply your client_secret."
    data = urlencode({
        'client_id': CLIENT_ID,
        'client_secret': CLIENT_SECRET,
        'grant_type': GRANT_TYPE,
    })
    headers = {
        'content-type': 'application/x-www-form-urlencoded',
    }
    response = requests.request('POST', url, data=data, headers=headers)
    bearer_token = response.json()['access_token']
    return bearer_token


def request(host, path, bearer_token, url_params=None):
    """Given a bearer token, send a GET request to the API.
    Args:
        host (str): The domain host of the API.
        path (str): The path of the API after the domain.
        bearer_token (str): OAuth bearer token, obtained using client_id and client_secret.
        url_params (dict): An optional set of query parameters in the request.
    Returns:
        dict: The JSON response from the request.
    Raises:
        HTTPError: An error occurs from the HTTP request.
    """
    url_params = url_params or {}
    url = '{0}{1}'.format(host, quote(path.encode('utf8')))
    headers = {
        'Authorization': 'Bearer %s' % bearer_token,
    }
    
    print(u'Querying {0} ...'.format(url))

    response = requests.request('GET', url, headers=headers, params=url_params)

    return response.json()


def search(bearer_token, term, location):
    """Query the Search API by a search term and location.
    Args:
        term (str): The search term passed to the API.
        location (str): The search location passed to the API.
    Returns:
        dict: The JSON response from the request.
    """

    url_params = {
        'term': term.replace(' ', '+'),
        'location': location.replace(' ', '+'),
        'limit': SEARCH_LIMIT
    }
    return request(API_HOST, SEARCH_PATH, bearer_token, url_params=url_params)


def get_business(bearer_token, business_id):
    """Query the Business API by a business ID.
    Args:
        business_id (str): The ID of the business to query.
    Returns:
        dict: The JSON response from the request.
    """
    business_path = BUSINESS_PATH + business_id

    return request(API_HOST, business_path, bearer_token)


def query_api(term, location):
    """Queries the API by the input values from the user.
    Args:
        term (str): The search term to query.
        location (str): The location of the business to query.
    """
    bearer_token = obtain_bearer_token(API_HOST, TOKEN_PATH)

    response = search(bearer_token, term, location)

    businesses = response.get('businesses')

    if not businesses:
        print(u'No businesses for {0} in {1} found.'.format(term, location))
        return

    business_id = businesses[0]['id']

    print(u'{0} businesses found, querying business info ' \
        'for the top result "{1}" ...'.format(
            len(businesses), business_id))
    response = get_business(bearer_token, business_id)

    print(u'Result for business "{0}" found:'.format(business_id))
    pprint.pprint(response, indent=2)


In [102]:
BEARER_TOKEN = obtain_bearer_token(API_HOST,TOKEN_PATH)

In [123]:
SEARCH_LIMIT = 50

In [147]:
search(BEARER_TOKEN,DEFAULT_TERM,DEFAULT_LOCATION).get('businesses')[0].get('name')


Querying https://api.yelp.com/v3/businesses/search ...
Calicraft Brewing


In [ ]:
DEFAULT_BASE_URI = "https://api.yelp.com/v3/businesses"
BASE_URI = ""
API_KEY = ""

TOKEN = "FD3xfq8PSpGKLnpjuH7efjRvuCQO8_5bChBRrjrbcqTgT1P9LfZPTMgDsoBKpiad22ZKO86f6tlaJZAvVJZwsaMoId9b9pXRkAyA31Io8PyTGQZLg6Q-EEQoOtF2WXYx"
TOKEN_TYPE = "BEARER"

simple_endpoints = ["id", "name", "review_count", "rating",
                    "coordinates", "price", "location"]

# single_param_endpoints = ["beer", "brewery", "category", "event",
#                          "feature", "glass", "guild", "ingredient",
#                          "location", "socialsite", "style", "menu"]

class Yelp:

    @staticmethod
    def __make_simple_endpoint_fun(name):
        @staticmethod
        def _function(options={}):
            return Yelp._get("/" + name, options)
        return _function

    @staticmethod
    def __make_singlearg_endpoint_fun(name):
        @staticmethod
        def _function(id, options={}):
            return Yelp._get("/" + name + "/" + id, options)
        return _function

    @staticmethod
    def _get(request, options):
        options.update({"key" : Yelp.API_KEY})
        r = requests.get(Yelp.BASE_URI + request, params=options)
        #print r.request.url
        return r.json()

    @staticmethod
    def configure(apikey, baseuri=DEFAULT_BASE_URI):
        Yelp.API_KEY = apikey
        Yelp.BASE_URI = baseuri
        for endpoint in simple_endpoints:
            fun = Yelp.__make_simple_endpoint_fun(endpoint)
            setattr(Yelp, endpoint.replace('/', '_'), fun)
        for endpoint in single_param_endpoints:
            fun = Yelp.__make_singlearg_endpoint_fun(endpoint)
            setattr(Yelp, endpoint.replace('/', '_'), fun)

In [ ]:
### Need to edit the code below ###

In [ ]:
# Extract Data
Yelp.configure(apikey)

In [ ]:
# Get Number of Pages

pages = BreweryDb.breweries()[u'numberOfPages']

# get breweries data
breweries = []
for page in range(1, pages+1):
    b = BreweryDb.breweries({'p': page})
    breweries.extend(b['data'])

In [ ]:
len(breweries)

In [ ]:
breweries = pd.DataFrame(breweries)
breweries.reset_index()

In [ ]:
breweries.to_csv('../../../data/brewery-db/breweries.csv', quoting=csv.QUOTE_NONNUMERIC, encoding='utf-8', index=False)

In [ ]:
beers = BreweryDb.beers({'p': 2, 'withBreweries': 'Y', 'withSocialAccounts': 'Y', 'withIngredients': 'Y'})

In [ ]:
beers

In [ ]:
# global variables
num_fetch_threads = 10
errors = []
queue = Queue()

# get number of pages
pages = BreweryDb.beers()[u'numberOfPages']

# build queue
for page in range(pages):
    q = page + 1
    queue.put(q)
    
# encode strings with utf-8
def encode_str(x):
    return str(x).encode('utf-8')

# encode all string columns
def clean_row(x):
    for c in x.keys():
        if isinstance(c, str):
            x[c] = encode_str(x[c])
    return x


# set up beers download progress bar
p = FloatProgress(min=0, max=queue.qsize(), description="Downloading beers")
display(p)

# define worker
def get_beers(i, q):
    while True:
        try:
            beers = []
            page = q.get() 
            # get beer data
            beers = BreweryDb.beers({'p': page, 'withBreweries': 'Y', 'withSocialAccounts': 'Y', 'withIngredients': 'Y'})
            with open('../../../data/brewery-db/%s.json' % page, 'w') as f:
                json.dump(beers, f, indent=4)
            
        except Exception as e: 
            errors.append(dict(page=page, error=str(e)))
            pass
        p.value += 1
        q.task_done()

# open threads
for i in range(num_fetch_threads):
    worker = Thread(target=get_beers, args=(i, queue,))
    worker.setDaemon(True)
    worker.start()

# start task queue
queue.join()

# send errors to csv
errors = pd.DataFrame(errors)
errors.to_csv('../../../data/brewery-db/beer-errors-1.csv', index=False, quoting=csv.QUOTE_NONNUMERIC)

print '*** Done'